 # Đồ án Cuối Kì Khoa học Dữ liệu 

********************************

# Import

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np


from requests_html import HTMLSession
from bs4 import BeautifulSoup
import json
import time # Dùng để sleep chương trình
import datetime as dt # Dùng để xử lý dữ liệu thời gian
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

In [3]:
import sklearn
sklearn.__version__

'0.23.1'

# Thu thập dữ liệu
- Nguồn dữ liệu: https://www.cars-data.com/en/

In [3]:
url = 'https://www.cars-data.com/en/all-cars.html'
Session = HTMLSession()
res = Session.get(url)
s = HTMLSession()

In [ ]:
link_car_urls=[] # List chứa các link xe chưa tính các mẫu của mỗi xe
for html in res.html: #Get link của các xe ở tất cả trang
    cars_one_page = html.find('section.models > div.col-4')
    #print(html.url)
    for car in cars_one_page: #Get link của các xe ở một trang
        url = car.find('a', first = True).attrs['href']
        link_car_urls.append(url)
        
df = pd.DataFrame({'urls': link_car_urls}) # Ghi vào file link_car_urls.csv
df.to_csv("link_car_urls.csv", sep = '\t')

In [ ]:
len(link_car_urls)


In [6]:
all_link_car_urls=[] #List chứa tất cả các xe bao gồm nhiều mẫu của từng xe
for car_url in link_car_urls:
    r = s.get(car_url)
    section_types = r.html.find('section.types', first = True)
    urls = section_types.find('h2 > a')
    for url in urls: #Duyệt vào trong từng link xe ở list car_urls
        if (url.attrs['href'] != ""):
            all_link_car_urls.append(url.attrs['href'])
    print(len(all_link_car_urls), end='\r')

In [7]:
len(all_link_car_urls)

45453

In [8]:
df = pd.DataFrame({'all_models_cars': all_link_car_urls})
df.to_csv("all_models_cars.csv", sep = '\t')

In [9]:
pd.set_option('display.max_colwidth', None)
all_cars=pd.read_csv('all_models_cars.csv', sep = '\t')['all_models_cars']
all_cars

0            https://www.cars-data.com/en/audi-rs4-avant-2.9-tfsi-quattro-specs/80342
1        https://www.cars-data.com/en/audi-rs5-sportback-2.9-tfsi-quattro-specs/80343
2                      https://www.cars-data.com/en/audi-tt-coupe-40-tfsi-specs/80385
3                      https://www.cars-data.com/en/audi-tt-coupe-45-tfsi-specs/80386
4                      https://www.cars-data.com/en/audi-tt-coupe-45-tfsi-specs/80387
                                             ...                                     
45448                            https://www.cars-data.com/en/lada-1200-e-specs/73776
45449                          https://www.cars-data.com/en/saab-99-gl-cm-specs/71469
45450                          https://www.cars-data.com/en/saab-99-gl-cm-specs/71473
45451                   https://www.cars-data.com/en/citroen-2cv-6-special-specs/9299
45452                      https://www.cars-data.com/en/citroen-2cv-6-club-specs/9300
Name: all_models_cars, Length: 45453, dtype: object

In [4]:
def get_detail_car_info(list_url, file_out_name):
    
    file_out = open(file_out_name, 'w', encoding='utf-8')
    file_out.write(f'url\tname\tmodel\tbrand\teLabel\tbodyType\tlength\theight\twidth\tweight\tweightTotal\temissionsCO2\tmodelDate\tfuelType\tnumberOfAxles\tnumberOfDoors\tnumberOfForwardGears\tseatingCapacity\tvehicleTransmission\tcargoVolume\troofLoad\taccelerationTime\tdriveWheelConfiguration\tfuelCapacity\tfuelConsumption\tspeed\tpayload\ttrailerWeight\tvEengineType\tvEfuelType\tvEengineDisplacement\tvEenginePower\ttorque\tprice\n')
    count=0
    for url in list_url:
        r = s.get(url)
        #print(r)
        # Get
        dts = r.html.find('dt.col-6')
        dds = r.html.find('dd.col-6')
        price = 0.0
        eLabel = 'NULL'
        length = 0.0
        for i in range(len(dts)):
            if (dts[i].text == 'price:'):
                price = dds[i].text.split()[-1]
            if (dts[i].text == 'energy label:'):
                eLabel = dds[i].text
            if (dts[i].text == 'length:'):
                length = dds[i].text.split()[0]

        soup = BeautifulSoup(r.html.html, 'html.parser')
        js = json.loads(soup.select_one("script[type='application/ld+json']").string)

        name = js.get('name', "NA")
        bodyType = js.get('bodyType', "NA")
        emissionsCO2 = js.get('emissionsCO2', "NA")
        modelDate = js.get('modelDate', "NA")
        fuelType = js.get('fuelType', "NA")
        numberOfAxles = js.get('numberOfAxles', "NA")
        numberOfDoors = js.get('numberOfDoors', "NA")
        numberOfForwardGears = js.get('numberOfForwardGears', "NA")
        seatingCapacity = js.get('seatingCapacity', "NA")
        vehicleTransmission = js.get('vehicleTransmission', "NA")
        model = js.get('model', "NA")
        url  = js.get('url', "NA")
        cargoVolume = js.get('cargoVolume', "NA")
        roofLoad = js.get('roofLoad', "NA")
        accelerationTime = js.get('accelerationTime', "NA")
        driveWheelConfiguration  = js.get('driveWheelConfiguration', {}).get('name', "NA")
        fuelCapacity = js.get('fuelCapacity', "NA")
        fuelConsumption = js.get('fuelConsumption', "NA")
        speed = js.get('speed', "NA")
        payload = js.get('payload', "NA")
        trailerWeight = js.get('trailerWeight', "NA")
        vEengineType = js.get('vehicleEngine', {}).get('engineType', "NA")
        vEfuelType = js.get('vehicleEngine', {}).get('fuelType', "NA")
        vEengineDisplacement = js.get('vehicleEngine', {}).get('engineDisplacement', "NA")
        vEenginePower = js.get('vehicleEngine', {}).get('enginePower', "NA")
        torque = js.get('vehicleEngine', {}).get('torque', "NA")
        weightTotal = js.get('weightTotal', "NA")
        wheelbase = js.get('wheelbase', "NA")
        height = js.get('height', "NA")
        brand = js.get('manufacturer', {}).get('name', "NA")
        weight = js.get('weight', "NA")
        width = js.get('width', "NA")
        file_out.write(f'{url}\t{name}\t{model}\t{brand}\t{eLabel}\t{bodyType}\t{length}\t{height}\t{width}\t{weight}\t{weightTotal}\t{emissionsCO2}\t{modelDate}\t{fuelType}\t{numberOfAxles}\t{numberOfDoors}\t{numberOfForwardGears}\t{seatingCapacity}\t{vehicleTransmission}\t{cargoVolume}\t{roofLoad}\t{accelerationTime}\t{driveWheelConfiguration}\t{fuelCapacity}\t{fuelConsumption}\t{speed}\t{payload}\t{trailerWeight}\t{vEengineType}\t{vEfuelType}\t{vEengineDisplacement}\t{vEenginePower}\t{torque}\t{price}\n')
        count+=1
        print(count,end='\r')
    file_out.close()

In [ ]:
get_detail_car_info(all_cars, 'all_cars.data')

# Bảng dữ liệu sau khi thu thập

In [4]:
cars = pd.read_csv("all_cars.data", sep = "\t")


C:\Users\Admin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Tiền xử lí dữ liệu

In [5]:
df = cars.copy()
df

,url,name,model,brand,eLabel,bodyType,length,height,width,weight,...,fuelConsumption,speed,payload,trailerWeight,vEengineType,vEfuelType,vEengineDisplacement,vEenginePower,torque,price
0,http://www.cars-data.com//en/audi-rs4-avant-2....,Audi RS4 Avant 2.9 TFSI quattro,Audi RS4 Avant,Audi,G,stationwagon,4781,1404 mm,1866 mm,1730 kg,...,"9,2 l/100km",250 km/h,620 kg,1900 kg,dohc,petrol,2894 cc,0 KW,600 Nm,122.000
1,http://www.cars-data.com//en/audi-rs5-sportbac...,Audi RS5 Sportback 2.9 TFSI quattro,Audi RS5 Sportback,Audi,G,hatchback,4783,1399 mm,1866 mm,1695 kg,...,"9,1 l/100km",250 km/h,625 kg,0 kg,dohc,petrol,2894 cc,0 KW,600 Nm,119.420
2,http://www.cars-data.com//en/audi-tt-coupe-40-...,Audi TT Coupe 40 TFSI,Audi TT Coupe,Audi,D,coupe,4191,1376 mm,1832 mm,1245 kg,...,"6,0 l/100km",250 km/h,460 kg,0 kg,dohc,petrol,1984 cc,0 KW,320 Nm,50.070
3,http://www.cars-data.com//en/audi-tt-coupe-45-...,Audi TT Coupe 45 TFSI,Audi TT Coupe,Audi,E,coupe,4191,1376 mm,1832 mm,1225 kg,...,"6,5 l/100km",250 km/h,460 kg,0 kg,dohc,petrol,1984 cc,0 KW,370 Nm,52.150
4,http://www.cars-data.com//en/audi-tt-coupe-45-...,Audi TT Coupe 45 TFSI,Audi TT Coupe,Audi,E,coupe,4191,1376 mm,1832 mm,1260 kg,...,"6,4 l/100km",250 km/h,460 kg,0 kg,dohc,petrol,1984 cc,0 KW,370 Nm,54.020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45448,http://www.cars-data.com//en/lada-1200-e-specs...,Lada 1200 E,Lada 1200,Lada,-,sedan,4073,1382 mm,1611 mm,934 kg,...,"9,5 l/100km",140 km/h,496 kg,750 kg,ohc,petrol,1198 cc,44 KW,89 Nm,4.152
45449,http://www.cars-data.com//en/saab-99-gl-cm-spe...,Saab 99 GL CM,Saab 99,Saab,N.A.,sedan,4480,1440 mm,1690 mm,1120 kg,...,"9,4 l/100km",160 km/h,440 kg,NaN,ohc,petrol,1985 cc,74 KW,162 Nm,11.340
45450,http://www.cars-data.com//en/saab-99-gl-cm-spe...,Saab 99 GL CM,Saab 99,Saab,N.A.,sedan,4480,1440 mm,1690 mm,1160 kg,...,"9,4 l/100km",160 km/h,450 kg,NaN,ohc,petrol,1985 cc,74 KW,162 Nm,11.794
45451,http://www.cars-data.com//en/citroen-2cv-6-spe...,Citroen 2CV 6 Special,Citroen 2CV,Citroen,N.A.,sedan,3830,1600 mm,1480 mm,560 kg,...,NaN,115 km/h,370 kg,400 kg,ohc,petrol,602 cc,21 KW,38 Nm,5.241


### Tách riêng hai dạng cột số và cat

### Kiểm tra có bao nhiêu dòng bị lặp trong dữ liệu

In [6]:
cars.duplicated().sum()

0

In [7]:
categorical_cols = ['url', 'name', 'model', 'brand', 'eLabel', 'bodyType', 'modelDate', 'fuelType', 'vehicleTransmission', 
                    'driveWheelConfiguration', 'vEengineType', 'vEfuelType'] #Phân loại cột kiểu Cat
number_cols = [] #Phân loại cột dạng Num
for x in cars.columns:
    if not (x in categorical_cols):
        number_cols.append(x)

assert (len(categorical_cols)+len(number_cols))==len(cars.columns) # Kiểm tra các cột tách ra có bằng số lượng cột ban đầu?

print('Caterogical Columns:', categorical_cols, '\n\n','Number Columns:', number_cols)


Caterogical Columns: ['url', 'name', 'model', 'brand', 'eLabel', 'bodyType', 'modelDate', 'fuelType', 'vehicleTransmission', 'driveWheelConfiguration', 'vEengineType', 'vEfuelType'] 

 Number Columns: ['length', 'height', 'width', 'weight', 'weightTotal', 'emissionsCO2', 'numberOfAxles', 'numberOfDoors', 'numberOfForwardGears', 'seatingCapacity', 'cargoVolume', 'roofLoad', 'accelerationTime', 'fuelCapacity', 'fuelConsumption', 'speed', 'payload', 'trailerWeight', 'vEengineDisplacement', 'vEenginePower', 'torque', 'price']


### Xử lí các cột dạng caterogical

In [8]:
for col in categorical_cols[:]:
    print(col, len(cars[col].unique()))

url 45453
name 25892
model 1677
brand 89
eLabel 9
bodyType 11
modelDate 51
fuelType 10
vehicleTransmission 1
driveWheelConfiguration 6
vEengineType 4
vEfuelType 10


In [9]:
(~(df['fuelType']==df['vEfuelType'])).sum()

100

1. Các cột url, name, model sẽ được loại bỏ vì không cần thiết<br>
2. Cột VehicleTransmission chỉ tồn tại duy nhất 1 giá trị, nên khi khai thác sẽ không có tác dụng<br>
3. Hai cột fuelType và vEfuelType khi so sánh ta thấy giá trị false (tức khác nhau giữa hai cột là 100 <<< 45310), nghĩa là hai cột này là giống nhau vì vậy xóa một cột, xóa vEfuelType vì fuelType tên ngắn hơn.

In [10]:
df=df.drop(columns=['url', 'name','model','vehicleTransmission','vEfuelType'])

In [171]:
df['eLabel'].unique() #Kiểm tra thấy tồn tại các giá trị 'N.A.' và giá trị '-'
def process_eLabel(x):
    if (x=='N.A.' or x=='-'):
        return None
    else:
        return x
df['eLabel'].unique()

array(['G', 'D', 'E', 'F', 'B', 'C', 'A', '-', 'N.A.'], dtype=object)

In [172]:
 df['bodyType'].unique()#Kiểm tra tồn tại giá trị nan nên sẽ drop

array(['stationwagon', 'hatchback', 'coupe', 'cabriolet', 'sedan',
       'suv/crossover', 'mpv', 'bus', 'pick-up', 'van', nan], dtype=object)

In [173]:
df['fuelType'].unique()#Tồn tại giá trị nan nên sẽ drop


array(['petrol', 'diesel', nan, 'LPG / petrol', 'aardgas / petrol',
       'aardgas', 'bio-ethanol', 'LPG', 'natural gas', 'benzine'],
      dtype=object)

In [174]:
df['driveWheelConfiguration'].unique() #Không có các giá trị nan

array(['front+rear', 'front', 'rear', 'achter', 'voor+achter', 'voor'],
      dtype=object)

In [175]:
df['vEengineType'].unique()#Tồn tại giá trị nan nên sẽ drop

array(['dohc', 'ohc', nan, 'ohv'], dtype=object)

In [176]:
df.dropna(subset=['fuelType','bodyType','eLabel'],inplace=True)
df

,brand,eLabel,bodyType,length,height,width,weight,weightTotal,emissionsCO2,modelDate,...,fuelCapacity,fuelConsumption,speed,payload,trailerWeight,vEengineType,vEengineDisplacement,vEenginePower,torque,price
0,Audi,G,stationwagon,4781,1404 mm,1866 mm,1730 kg,1730 kg,208 g/km,2019,...,58 l,"9,2 l/100km",250 km/h,620 kg,1900 kg,dohc,2894 cc,0 KW,600 Nm,122.000
1,Audi,G,hatchback,4783,1399 mm,1866 mm,1695 kg,1695 kg,207 g/km,2019,...,58 l,"9,1 l/100km",250 km/h,625 kg,0 kg,dohc,2894 cc,0 KW,600 Nm,119.420
2,Audi,D,coupe,4191,1376 mm,1832 mm,1245 kg,1245 kg,137 g/km,2019,...,50 l,"6,0 l/100km",250 km/h,460 kg,0 kg,dohc,1984 cc,0 KW,320 Nm,50.070
3,Audi,E,coupe,4191,1376 mm,1832 mm,1225 kg,1225 kg,149 g/km,2019,...,50 l,"6,5 l/100km",250 km/h,460 kg,0 kg,dohc,1984 cc,0 KW,370 Nm,52.150
4,Audi,E,coupe,4191,1376 mm,1832 mm,1260 kg,1260 kg,147 g/km,2019,...,50 l,"6,4 l/100km",250 km/h,460 kg,0 kg,dohc,1984 cc,0 KW,370 Nm,54.020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45448,Lada,-,sedan,4073,1382 mm,1611 mm,934 kg,934 kg,- g/km,1971,...,39 l,"9,5 l/100km",140 km/h,496 kg,750 kg,ohc,1198 cc,44 KW,89 Nm,4.152
45449,Saab,N.A.,sedan,4480,1440 mm,1690 mm,1120 kg,1120 kg,NaN,1971,...,58 l,"9,4 l/100km",160 km/h,440 kg,NaN,ohc,1985 cc,74 KW,162 Nm,11.340
45450,Saab,N.A.,sedan,4480,1440 mm,1690 mm,1160 kg,1160 kg,NaN,1971,...,58 l,"9,4 l/100km",160 km/h,450 kg,NaN,ohc,1985 cc,74 KW,162 Nm,11.794
45451,Citroen,N.A.,sedan,3830,1600 mm,1480 mm,560 kg,560 kg,NaN,1970,...,25 l,NaN,115 km/h,370 kg,400 kg,ohc,602 cc,21 KW,38 Nm,5.241


### Xử lí các cột dạng số

In [11]:
# Xóa các dòng mà price = nan
df = df[df.price != 'N.A.']
df['price'].unique()

array(['122.000', '119.420', '50.070', ..., '4.152', '5.241', '5.704'],
      dtype=object)

In [12]:
def prepro_num(num): #Xử lí các cột số
    if type(num) == str: #Có col có dạng int thì giữ nguyên
        num = num.replace(',','.') #hỗ trợ chuyển string sang float vì , thì không thể chuyển được
        num = num.split()[0] #Tách phần sau ra như đơn vị: km, m, ...
    val = None # Dùng biến tạp để ép kiểu sang float cho các giá trị trên
    try:
        val = float(num)
    except ValueError:
        return val
    return val

for x in number_cols: #Lặp các cột trong các cột dạng số để lọc
        df[x] = df[x].apply(prepro_num)

In [14]:
df.replace('N.A.',np.nan, inplace = True)
df.replace('-',np.nan, inplace = True)

In [15]:
df

,brand,eLabel,bodyType,length,height,width,weight,weightTotal,emissionsCO2,modelDate,...,fuelCapacity,fuelConsumption,speed,payload,trailerWeight,vEengineType,vEengineDisplacement,vEenginePower,torque,price
0,Audi,G,stationwagon,4781.0,1404.0,1866.0,1730.0,1730.0,208.0,2019,...,58.0,9.2,250.0,620.0,1900.0,dohc,2894.0,0.0,600.0,122.000
1,Audi,G,hatchback,4783.0,1399.0,1866.0,1695.0,1695.0,207.0,2019,...,58.0,9.1,250.0,625.0,0.0,dohc,2894.0,0.0,600.0,119.420
2,Audi,D,coupe,4191.0,1376.0,1832.0,1245.0,1245.0,137.0,2019,...,50.0,6.0,250.0,460.0,0.0,dohc,1984.0,0.0,320.0,50.070
3,Audi,E,coupe,4191.0,1376.0,1832.0,1225.0,1225.0,149.0,2019,...,50.0,6.5,250.0,460.0,0.0,dohc,1984.0,0.0,370.0,52.150
4,Audi,E,coupe,4191.0,1376.0,1832.0,1260.0,1260.0,147.0,2019,...,50.0,6.4,250.0,460.0,0.0,dohc,1984.0,0.0,370.0,54.020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45448,Lada,NaN,sedan,4073.0,1382.0,1611.0,934.0,934.0,NaN,1971,...,39.0,9.5,140.0,496.0,750.0,ohc,1198.0,44.0,89.0,4.152
45449,Saab,NaN,sedan,4480.0,1440.0,1690.0,1120.0,1120.0,NaN,1971,...,58.0,9.4,160.0,440.0,NaN,ohc,1985.0,74.0,162.0,11.340
45450,Saab,NaN,sedan,4480.0,1440.0,1690.0,1160.0,1160.0,NaN,1971,...,58.0,9.4,160.0,450.0,NaN,ohc,1985.0,74.0,162.0,11.794
45451,Citroen,NaN,sedan,3830.0,1600.0,1480.0,560.0,560.0,NaN,1970,...,25.0,NaN,115.0,370.0,400.0,ohc,602.0,21.0,38.0,5.241


### Chia tập train, validation, test (60%, 20%, 20%)

In [16]:
# Tách X và y
y_price = df["price"] # sr là viết tắt của series
X_df = df.drop("price", axis=1)

In [17]:
train_X_df, val_X_df, train_y_price, val_y_price = train_test_split(X_df, y_price, test_size=0.4, random_state=0)

In [18]:
val_X_df, test_X_df, val_y_price, test_y_price = train_test_split(val_X_df, val_y_price, test_size=0.5, random_state=0)

In [19]:
print(train_X_df.shape)
print(val_X_df.shape)
print(test_X_df.shape)

(27186, 28)
(9062, 28)
(9062, 28)


### Tạo pipeline

In [20]:
def f(x, l):
    if type(x) is list:
        l.update(x)
class MultilabelEncoding(BaseEstimator, TransformerMixin):
    def __init__(self, col_name):
        self.col_name = col_name
        self.type_list = set()
    def fit(self, X_df, y=None):
        types_sr = X_df[self.col_name].str.replace('[/+]', ' ').str.split()
        types_sr.apply(f, args=(self.type_list,))
        self.type_list = list(self.type_list)
        return self
    def transform(self, X_df, y=None):
        transformed_df = X_df.copy()
        for i in range(len(self.type_list)):
            transformed_df[self.col_name + '_' + self.type_list[i]] = transformed_df[self.col_name].apply(lambda x: 1 if ((type(x) is str) and (self.type_list[i] in x)) else 0)
        transformed_df.drop(self.col_name, axis=1, inplace=True)
        return transformed_df

In [21]:
driveWheelConfig_encoder = MultilabelEncoding('driveWheelConfiguration')
df['driveWheelConfiguration']

0        front+rear
1        front+rear
2             front
3             front
4             front
            ...    
45448          rear
45449         front
45450         front
45451         front
45452         front
Name: driveWheelConfiguration, Length: 45310, dtype: object

In [16]:
#col_adderdropper = FunctionTransformer(ColAdderDropper)

In [17]:
#a = col_adderdropper.fit_transform(train_X_df)
#a['eLabel']

In [23]:
number_cols = ['length', 'height', 'width', 'weight', 'weightTotal', 'emissionsCO2', 'numberOfAxles', 'numberOfDoors', 
            'numberOfForwardGears', 'seatingCapacity', 'roofLoad', 'accelerationTime', 'fuelCapacity', 
            'fuelConsumption', 'speed', 'payload', 'trailerWeight', 'vEengineDisplacement', 'vEenginePower', 'torque']

categorical_cols = ['brand', 'bodyType', 'modelDate', 'fuelType', 'vehicleTransmission', 
                    'driveWheelConfiguration', 'vEengineType', 'vEfuelType','eLabel']

mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

mode_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

column_transformer = ColumnTransformer([('num_cols', mean_imputer, number_cols),
                                    ('categorical_cols', mode_imputer, categorical_cols)])

col_standardized = StandardScaler()

mlpclassifier = MLPClassifier(hidden_layer_sizes=(20), activation='tanh',
                              solver='lbfgs', random_state=0, max_iter=2500)

full_pipeline = make_pipeline(driveWheelConfig_encoder, column_transformer)
#full_pipeline = make_pipeline(col_adderdropper, column_transformer, col_standardized, mlpclassifier)



In [25]:
train_X_df

,brand,eLabel,bodyType,length,height,width,weight,weightTotal,emissionsCO2,modelDate,...,driveWheelConfiguration,fuelCapacity,fuelConsumption,speed,payload,trailerWeight,vEengineType,vEengineDisplacement,vEenginePower,torque
42066,Ford,NaN,sedan,4467.0,1407.0,1698.0,1050.0,1050.0,NaN,1987,...,rear,60.0,8.5,178.0,550.0,1350.0,dohc,1993.0,74.0,153.0
1395,Alfa Romeo,D,suv/crossover,4687.0,1671.0,1903.0,1635.0,1635.0,161.0,2017,...,front+rear,64.0,7.0,230.0,665.0,2300.0,dohc,1995.0,206.0,400.0
20999,Volkswagen,G,coupe,4256.0,1404.0,1810.0,1295.0,1295.0,145.0,2008,...,front,50.0,5.5,205.0,475.0,NaN,dohc,1968.0,103.0,320.0
19735,Citroen,F,sedan,4779.0,1458.0,1860.0,1485.0,1485.0,188.0,2008,...,front,71.0,7.9,200.0,530.0,1530.0,dohc,1749.0,92.0,170.0
41804,Mitsubishi,NaN,sedan,4235.0,1405.0,1670.0,1020.0,1020.0,NaN,1988,...,front,50.0,8.0,156.0,445.0,1000.0,ohc,1468.0,62.0,122.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30410,Land Rover,G,suv/crossover,4382.0,1757.0,1805.0,1427.0,1427.0,248.0,2002,...,front+rear,59.0,10.4,170.0,613.0,2000.0,dohc,1796.0,86.0,160.0
21247,BMW,G,cabriolet,4580.0,1384.0,1782.0,1725.0,1725.0,170.0,2007,...,rear,61.0,6.5,245.0,455.0,1800.0,dohc,2993.0,170.0,500.0
42641,Suzuki,NaN,hatchback,3770.0,1350.0,1540.0,750.0,750.0,NaN,1986,...,front,33.0,5.4,163.0,380.0,800.0,ohc,1324.0,49.0,104.0
43603,Toyota,NaN,stationwagon,4190.0,1390.0,1610.0,890.0,890.0,NaN,1984,...,rear,47.0,7.0,155.0,510.0,1000.0,ohc,1290.0,48.0,98.0


In [24]:
full_pipeline.fit_transform(train_X_df)


ValueError: A given column is not a column of the dataframe

In [34]:
#Có mấy cái giá trị thì nó bị nan sau khi clean nhưng mà ít quá thì thôi bỏ qua có cargoVolume nó nhiều quá nên xử lí riêng cho nó để nó làm sạch
#Nó bị Nan ở những chỗ mà giá trị là khoảng 500-1000 l còn mấy chỗ giá trị cố định thì không lỗi
No_Clean=df.isnull().sum()
No_Clean

url                            0
name                           0
model                          0
brand                          0
eLabel                      8916
bodyType                     402
length                        29
height                        60
width                         36
weight                       163
weightTotal                  163
emissionsCO2                8916
modelDate                      0
fuelType                     100
numberOfAxles                  0
numberOfDoors                  0
numberOfForwardGears        1093
seatingCapacity             1181
vehicleTransmission            0
cargoVolume                34026
roofLoad                    8123
accelerationTime            1543
driveWheelConfiguration        0
fuelCapacity                 119
fuelConsumption              778
speed                        461
payload                      814
trailerWeight               2712
vEengineType                  97
vEfuelType                   100
vEengineDi

In [35]:
Clean=df.isnull().sum()
Clean

url                            0
name                           0
model                          0
brand                          0
eLabel                      8916
bodyType                     402
length                        29
height                        60
width                         36
weight                       163
weightTotal                  163
emissionsCO2                8916
modelDate                      0
fuelType                     100
numberOfAxles                  0
numberOfDoors                  0
numberOfForwardGears        1093
seatingCapacity             1181
vehicleTransmission            0
cargoVolume                34026
roofLoad                    8123
accelerationTime            1543
driveWheelConfiguration        0
fuelCapacity                 119
fuelConsumption              778
speed                        461
payload                      814
trailerWeight               2712
vEengineType                  97
vEfuelType                   100
vEengineDi

0